In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [2]:
df = pd.read_csv("sales_pos.csv")
df.head(2)

,user,prod,gender,age_group,job,city,marital,prod_cat1,prod_cat2,prod_cat3,purchase
0,1,P00069042,F,0-17,10,A,0,3,NaN,NaN,8370
1,1,P00248942,F,0-17,10,A,0,1,6.0,14.0,15200


### Q1.

In [3]:
df_q1 = df[["prod", "job", "purchase"]].copy()
df_q1.head(2)

,prod,job,purchase
0,P00069042,10,8370
1,P00248942,10,15200


In [4]:
df_q1.groupby("prod")["purchase"].sum().max()

27995166

In [5]:
top_prod = df_q1.groupby("prod")["purchase"].sum().idxmax()
top_prod

'P00025442'

In [6]:
df_q1_sub = df_q1.loc[df_q1["prod"] == top_prod, ]
df_q1_sub.head(2)

,prod,job,purchase
667,P00025442,17,19706
749,P00025442,7,15212


In [7]:
df_q1_sub["job"].value_counts().idxmax()

4

### Q2.

In [8]:
df_u1 = df.loc[df["user"] == 1, ["prod_cat1", "prod_cat2", "prod_cat3"]].copy()
df_u1 = df_u1.fillna(0)

In [9]:
df_u1.shape

(35, 3)

In [10]:
df_u1.sort_values(df_u1.columns.to_list())

,prod_cat1,prod_cat2,prod_cat3
78145,1,2.0,5.0
428893,1,2.0,9.0
1,1,6.0,14.0
194558,1,8.0,17.0
39180,2,4.0,8.0
0,3,0.0,0.0
311710,3,4.0,0.0
311712,3,4.0,0.0
311713,3,4.0,0.0
116851,3,4.0,5.0


In [11]:
df_u1.drop_duplicates().shape

(21, 3)

In [12]:
df_u1["prod_cat1"] = df_u1["prod_cat1"].astype("str")
df_u1["prod_cat2"] = df_u1["prod_cat2"].astype("int").astype("str")
df_u1["prod_cat3"] = df_u1["prod_cat3"].astype("int").astype("str")
df_u1["prod_cat" ] = df_u1["prod_cat1"] + "-" + df_u1["prod_cat2"] + "-" + df_u1["prod_cat3"]

In [13]:
df_u1["prod_cat"].nunique()

21

In [14]:
df["age_group"].unique()

array(['0-17', '55+', '26-35', '46-50', '51-55', '36-45', '18-25'],
      dtype=object)

In [15]:
df_q2 = df.loc[df["age_group"] == "26-35", ["user", "marital", "prod_cat1", "prod_cat2", "prod_cat3"]]
df_q2 = df_q2.reset_index(drop = True)
df_q2 = df_q2.fillna(0)

메모장에서 \[Ctrl\] + \[h\] 하면은 일괄 바꾸기 가능

In [16]:
df_q2["prod_cat1"] = df_q2["prod_cat1"].astype("str")
df_q2["prod_cat2"] = df_q2["prod_cat2"].astype("int").astype("str")
df_q2["prod_cat3"] = df_q2["prod_cat3"].astype("int").astype("str")
df_q2["prod_cat" ] = df_q2["prod_cat1"] + "-" + df_q2["prod_cat2"] + "-" + df_q2["prod_cat3"]

In [17]:
# df_q2.groupby("user")["prod_cat"].nunique()
# 사용자 id와 결혼정보를 중복제거하여 join 하는 방식 대신 groupby 를 활용하면 됨.
df_q2_g = df_q2.groupby(["user", "marital"])["prod_cat"].nunique().reset_index()
df_q2_g.head()

,user,marital,prod_cat
0,3,0,18
1,5,1,43
2,8,1,32
3,9,0,31
4,11,0,34


In [18]:
stat_m0 = df_q2_g.loc[df_q2_g["marital"] == 0, "prod_cat"].mean()
stat_m1 = df_q2_g.loc[df_q2_g["marital"] == 1, "prod_cat"].mean()
stat_m0, stat_m1

(41.66318327974277, 41.79233621755253)

In [19]:
round(abs(stat_m0 - stat_m1), 2)

0.13

### Q3.

In [20]:
df.head(1)

,user,prod,gender,age_group,job,city,marital,prod_cat1,prod_cat2,prod_cat3,purchase
0,1,P00069042,F,0-17,10,A,0,3,NaN,NaN,8370


In [21]:
col_user = ["user", "gender", "age_group", "job", "city", "marital"]
df_user = df.loc[:, col_user].drop_duplicates().reset_index(drop = True)
df_user.head(2)

,user,gender,age_group,job,city,marital
0,1,F,0-17,10,A,0
1,2,M,55+,16,C,0


In [22]:
df["user"].nunique(), len(df_user)

(5891, 5891)

In [23]:
df_g = df.groupby("user")[["prod", "purchase"]].agg({"prod": "nunique", 
                                                     "purchase": "sum"})
df_g = df_g.reset_index()
df_g.head()

,user,prod,purchase
0,1,35,334093
1,2,77,810472
2,3,29,341635
3,4,14,206468
4,5,106,821001


In [24]:
df_q3 = df_user.merge(df_g, on = "user")
df_q3.head(2)

,user,gender,age_group,job,city,marital,prod,purchase
0,1,F,0-17,10,A,0,35,334093
1,2,M,55+,16,C,0,77,810472


In [25]:
col_user = ["user", "gender", "age_group", "job", "city", "marital"]
df_q3 = df.groupby(col_user)[["prod", "purchase"]].agg({"prod": "nunique", 
                                                        "purchase": "sum"})
df_q3 = df_q3.reset_index()
df_q3.head(2)

,user,gender,age_group,job,city,marital,prod,purchase
0,1,F,0-17,10,A,0,35,334093
1,2,M,55+,16,C,0,77,810472


In [26]:
df_q3["gender"] = df_q3["gender"].replace({"M": 1, "F": 0})

In [27]:
arr_u = df_q3["age_group"].unique()
# arr_u = arr_u.sort() # Array 이렇게 작성하지 않음.
arr_u.sort()
arr_u

array(['0-17', '18-25', '26-35', '36-45', '46-50', '51-55', '55+'],
      dtype=object)

In [28]:
ser_repl = pd.Series(range(7), index = arr_u)
ser_repl

0-17     0
18-25    1
26-35    2
36-45    3
46-50    4
51-55    5
55+      6
dtype: int64

In [29]:
pd.crosstab(df_q3["age_group"], df_q3["age_group"].replace(ser_repl))

age_group,0,1,2,3,4,5,6
age_group,,,,,,,
0-17,218,0,0,0,0,0,0
18-25,0,1069,0,0,0,0,0
26-35,0,0,2053,0,0,0,0
36-45,0,0,0,1167,0,0,0
46-50,0,0,0,0,531,0,0
51-55,0,0,0,0,0,481,0
55+,0,0,0,0,0,0,372


In [30]:
df_q3["age_group"] = df_q3["age_group"].replace(ser_repl)

In [31]:
# df_dum = pd.get_dummies(df_q3, columns = ["job", "city"]) # 시험버전
df_dum = pd.get_dummies(df_q3, columns = ["job", "city"], dtype = "int")
df_dum = df_dum.drop(columns = "user")
df_dum.head(2)

,gender,age_group,marital,prod,purchase,job_0,job_1,job_2,job_3,job_4,...,job_14,job_15,job_16,job_17,job_18,job_19,job_20,city_A,city_B,city_C
0,0,0,0,35,334093,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,6,0,77,810472,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [32]:
arr_dum_nor = MinMaxScaler().fit_transform(df_dum)

C:\Users\Encaion\anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\Encaion\anaconda3\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\Encaion\anaconda3\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\Encaion\anaconda3\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` inst

In [33]:
arr_dum_nor[:1, ]

array([[0.        , 0.        , 0.        , 0.02843137, 0.02739807,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        ]])

In [34]:
model_kmeans = KMeans(n_clusters = 7, random_state = 123)
model_kmeans.fit(arr_dum_nor)

C:\Users\Encaion\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=7, random_state=123)

In [35]:
round(silhouette_score(arr_dum_nor, labels = model_kmeans.labels_), 2)

0.18